# **Analyse Exploratoire**

In [1]:
# Bibliothèques de base
import pandas as pd
import numpy as np

# Visualisations
import seaborn as sns
import matplotlib.pyplot as plt

# Utilisés lors de l'ACP
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Servira pour faire l'ANOVA
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Pour faire tests de normalité
from scipy.stats import shapiro

# Utilisé pour passer des données au log avant ANOVA
from sklearn.preprocessing import FunctionTransformer

# Utilisé pour savoir si le fichier excel d'export existe déjà
import os

# Je désactive le copy warning, en m'étant au préalable assuré que les traitements étaient corrects
# pd.options.mode.chained_assignment = None  # default='warn'

## **Importation des datasets**

In [2]:
customers = pd.read_csv("dataset\olist_customers_dataset.csv", sep=',')
geolocation = pd.read_csv("dataset\olist_geolocation_dataset.csv", sep=',')
order_items = pd.read_csv("dataset\olist_order_items_dataset.csv", sep=',')
order_payments = pd.read_csv("dataset\olist_order_payments_dataset.csv", sep=',')
order_reviews = pd.read_csv("dataset\olist_order_reviews_dataset.csv", sep=',')
orders = pd.read_csv("dataset\olist_orders_dataset.csv", sep=',')
products = pd.read_csv("dataset\olist_products_dataset.csv", sep=',')
sellers = pd.read_csv("dataset\olist_sellers_dataset.csv", sep=',')
product_category_name_translation = pd.read_csv("dataset\product_category_name_translation.csv", sep=',')

print('Importation terminée')

Importation terminée


## **Pré-traitement des datasets avant fusion**

### **Traitement de *geolocation***

Je supprime *geolocation_city* et *geolocation_state* que je ne vais pas utiliser.<br>
Je regroupe par *geolocation_zip_code_prefix* en faisant la moyen pour les coordonnées GPS.

In [3]:
print('Lignes avant traitement :', len(geolocation))
geolocation = geolocation.drop(['geolocation_city', 'geolocation_state'], axis=1)
geolocation = geolocation.groupby('geolocation_zip_code_prefix').mean()
geolocation = geolocation.reset_index() # pour remettre geolocation_zip_code_prefix en tant que colonne
print('Lignes après traitement :', len(geolocation))
print('Valeurs manquantes :', geolocation.isnull().sum().sum())

Lignes avant traitement : 1000163
Lignes après traitement : 19015
Valeurs manquantes : 0


### **Traitement de *customers***

Je fusionne avec *geolocation* pour récupérer les coordonnées GPS à partir du zip code prefix<br>
Je renomme *geolocation_lat* et *geolocation_lng* en *customer_lat* et *customer_lng*<br>
Je ne conserve au final que *customer_id*, *customer_unique_id*, *customer_lat* et *customer_lng*

In [4]:
customers = customers.merge(geolocation, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
customers.rename(columns={"geolocation_lat": "customer_lat", "geolocation_lng": "customer_lng"}, inplace=True)
customers = customers[['customer_id', 'customer_unique_id', 'customer_lat', 'customer_lng']]
# Il y a quelques valeurs manquantes dans customer_lat/customer_lng : j'impute par la moyenne
customers['customer_lat'].loc[customers['customer_lat'].isnull() == True] = customers['customer_lat'].mean()
customers['customer_lng'].loc[customers['customer_lng'].isnull() == True] = customers['customer_lng'].mean()
print('Valeurs manquantes :', customers.isnull().sum().sum())

Valeurs manquantes : 0


### **Traitement de *sellers***

Je fusionne avec *geolocation* pour récupérer les coordonnées GPS à partir du zip code prefix<br>
Je renomme *geolocation_lat* et *geolocation_lng* en *seller_lat* et *seller_lng*<br>
Je ne conserve au final que *seller_id*, *seller_lat*, et *seller_lng*

In [5]:
sellers = sellers.merge(geolocation, left_on='seller_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')
sellers.rename(columns={"geolocation_lat": "seller_lat", "geolocation_lng": "seller_lng"}, inplace=True)
sellers = sellers[['seller_id', 'seller_lat', 'seller_lng']]
# Il y a quelques valeurs manquantes dans seller_lat/seller_lng : j'impute par la moyenne
sellers['seller_lat'].loc[sellers['seller_lat'].isnull() == True] = sellers['seller_lat'].mean()
sellers['seller_lng'].loc[sellers['seller_lng'].isnull() == True] = sellers['seller_lng'].mean()
print('Valeurs manquantes :', sellers.isnull().sum().sum())

Valeurs manquantes : 0


### **Traitement de *order_items***

Je regroupe par numéro de commande (order_id), les autres colonnes sont agglomérées comme suit : 
- *order_item_id* : *max()*, ce qui donnera le nombre d'articles de la commande
- *product_id* : *first()*, je ne conserve que le premier, pas idéal mais ça ne concerne qu'un nombre très limité de commandes
- *seller_id* : *first()*, même logique que pour *product_id*
- *shipping_limit_date* : *first()*, c'est la même pour toute la commande
- *price* : *sum()*, prix total des articles
- *freight_value* : *mean()*, frais de port moyens

In [6]:
print('Lignes avant traitement :', len(order_items))
order_items = order_items.groupby('order_id').agg({'order_item_id': 'max', 'product_id': 'first', 'seller_id': 'first', 'shipping_limit_date': 'first', 'price': 'sum', 'freight_value': 'mean'})
order_items = order_items.reset_index() # pour remettre order_id en tant que colonne
print('Lignes après traitement :', len(order_items))
print('Valeurs manquantes :', order_items.isnull().sum().sum())

Lignes avant traitement : 112650
Lignes après traitement : 98666
Valeurs manquantes : 0


### **Traitement de *order_payments***

Je supprime *payment_sequential*, que je ne vais pas utiliser.<br>
Je regroupe par *order_id*, les autres colonnes sont agglomérées comme suit : 
- *payment_type* : *first()*
- *payment_installments* (paiements en plusieurs fois) : *max()*
- *payment_value* : *sum()*

In [7]:
print('Lignes avant traitement :', len(order_payments))
order_payments = order_payments.drop(['payment_sequential'], axis=1)
order_payments = order_payments.groupby('order_id').agg({'payment_type': 'first', 'payment_installments': 'max', 'payment_value': 'sum'})
order_payments = order_payments.reset_index() # pour remettre order_id en tant que colonne
print('Lignes après traitement :', len(order_payments))
print('Valeurs manquantes :', order_payments.isnull().sum().sum())

Lignes avant traitement : 103886
Lignes après traitement : 99440
Valeurs manquantes : 0


### **Traitement de *order_reviews***

Je supprime *review_id*, *review_comment_title*, *review_comment_message*, *review_creation_date* et *review_answer_timestamp*, que je n'utiliserai pas.<br>
Je regroupe par *order_id* avec *mean()* sur *review_score* (il arrive parfois qu'il y ait plusieurs avis pour une même commande)

In [8]:
print('Lignes avant traitement :', len(order_reviews))
order_reviews = order_reviews.drop(['review_id', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'], axis=1)
order_reviews = order_reviews.groupby('order_id').mean()
order_reviews = order_reviews.reset_index() # pour remettre order_id en tant que colonne
print('Lignes après traitement :', len(order_reviews))
print('Valeurs manquantes :', order_reviews.isnull().sum().sum())

Lignes avant traitement : 99224
Lignes après traitement : 98673
Valeurs manquantes : 0


### **Traitement de *orders***

Je convertis les colonnes contenant des dates que je vais utiliser au format date.<br>
Je décide de ne conserver que les commandes qui sont indiquées comme livrées (la grande majorité)<br>
Je crée une nouvelle colonne *delivery_time* en faisant la différence entre *order_purchase_timestamp* et *order_delivered_customer_date*.<br>
Au final, je ne conserve que *order_id*, *customer_id*, *order_purchase_timestamp* et *delivery_time*



In [9]:
print('Lignes avant traitement :', len(orders))
# je passe par .apply() pour appliquer pd.to_datetime au deux colonnes
orders[['order_purchase_timestamp', 'order_delivered_customer_date']] = orders[['order_purchase_timestamp', 'order_delivered_customer_date']].apply(pd.to_datetime, format='%Y-%m-%d %H:%M:%S', errors='coerce')
orders = orders.loc[orders['order_status'] == 'delivered']
orders['delivery_time'] = (orders['order_delivered_customer_date'] - orders['order_purchase_timestamp']).dt.days
orders = orders[['order_id', 'customer_id', 'order_purchase_timestamp', 'delivery_time']]
# Je supprime les quelques lignes où delivery_time est manquant
orders = orders.loc[orders['delivery_time'].isnull() == False]
print('Lignes après traitement :', len(orders))
print('Valeurs manquantes :', orders.isnull().sum().sum())


Lignes avant traitement : 99441
Lignes après traitement : 96470
Valeurs manquantes : 0


### **Traitement de *products***

Je fusionne avec *product_category_name_translation* pour avoir les catégories en Anglais.<br>
Je remplace les valeurs manquantes dans *product_category_name_english* par "other".<br>
Au final, je ne conserve que les colonnes *product_id* et *product_category_name_english*.

In [10]:
products = products.merge(product_category_name_translation, on='product_category_name', how='left')
products['product_category_name_english'].loc[products['product_category_name_english'].isnull() == True] = "other"
products = products[['product_id', 'product_category_name_english']]
print('Valeurs manquantes :', products.isnull().sum().sum())

Valeurs manquantes : 0


C:\Users\Data Science\AppData\Local\Temp\ipykernel_5604\1600838163.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  products['product_category_name_english'].loc[products['product_category_name_english'].isnull() == True] = "other"


## **Fusion des datasets**

Je pars de *order_items* et je vais successivement fusionner avec les autres datasets.

In [11]:
order_items = order_items.merge(orders, on='order_id', how='left')
order_items = order_items.merge(products, on='product_id', how='left')
order_items = order_items.merge(order_payments, on='order_id', how='left')
order_items = order_items.merge(sellers, on='seller_id', how='left')
order_items = order_items.merge(order_reviews, on='order_id', how='left')
order_items = order_items.merge(customers, on='customer_id', how='left')

In [12]:
order_items

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,customer_id,order_purchase_timestamp,delivery_time,product_category_name_english,payment_type,payment_installments,payment_value,seller_lat,seller_lng,review_score,customer_unique_id,customer_lat,customer_lng
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29,3ce436f183e68e07877b285a838db11a,2017-09-13 08:59:02,7.0,cool_stuff,credit_card,2.0,72.19,-22.496953,-44.127492,5.0,871766c5855e863f6eccc05f988b23cb,-21.762775,-41.309633
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93,f6dd3ec061db4e3987629fe6b26e5cce,2017-04-26 10:53:06,16.0,pet_shop,credit_card,3.0,259.83,-23.565096,-46.518565,4.0,eb28e67c4c0b83846050ddfb8a35d051,-20.220527,-50.903424
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87,6489ae5e4333f3693df5ad4372dab6d3,2018-01-14 14:33:31,7.0,furniture_decor,credit_card,5.0,216.87,-22.262584,-46.171124,5.0,3818d81c6709e39d06b2738a8d3a2474,-19.870305,-44.593326
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79,d4eb9395c8c0431ee92fce09860c5a06,2018-08-08 10:00:35,6.0,perfumery,credit_card,2.0,25.78,-20.553624,-47.387359,4.0,af861d436cfc08b2c2ddefd0ba074622,-23.089925,-46.611654
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14,58dbd0b2d70206bf40e62cd34e84d795,2017-02-04 13:57:51,25.0,garden_tools,credit_card,3.0,218.04,-22.929384,-53.135873,5.0,64b576fb70d441e8f1b2d7d446e483c5,-23.243402,-46.827614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98661,fffc94f6ce00a00581880bf54a75a037,1,4aa6014eceb682077f9dc4bffebc05b0,b8bc237ba3788b23da09c0f1f3a3288c,2018-05-02 04:11:01,299.99,43.41,b51593916b4b8e0d6f66f2ae24f2673d,2018-04-23 13:57:06,17.0,housewares,boleto,1.0,343.40,-26.912574,-48.673980,5.0,0c9aeda10a71f369396d0c04dce13a64,-2.497993,-44.297761
98662,fffcd46ef2263f404302a634eb57f7eb,1,32e07fd915822b0765e448c4dd74c828,f3c38ab652836d21de61fb8314b69182,2018-07-20 04:31:48,350.00,36.53,84c5d4fbaf120aae381fad077416eaa0,2018-07-14 10:26:46,9.0,computers_accessories,boleto,1.0,386.53,-23.535864,-46.642819,5.0,0da9fe112eae0c74d3ba1fe16de0988b,-25.566904,-49.309115
98663,fffce4705a9662cd70adb13d4a31832d,1,72a30483855e2eafc67aee5dc2560482,c3cfdc648177fdbbbb35635a37472c53,2017-10-30 17:14:25,99.90,16.95,29309aa813182aaddc9b259e31b870e6,2017-10-23 17:07:56,4.0,sports_leisure,credit_card,3.0,116.85,-25.469955,-49.289821,5.0,cd79b407828f02fdbba457111c38e4c4,-23.597794,-46.643923
98664,fffe18544ffabc95dfada21779c9644f,1,9c422a519119dcad7575db5af1ba540e,2b3e4a2a3ea8e01938cabda2a3e5cc79,2017-08-21 00:04:32,55.99,8.72,b5e6afd5a41800fdf401e0272ca74655,2017-08-14 23:02:59,1.0,computers_accessories,credit_card,3.0,64.71,-23.635530,-46.694031,5.0,eb803377c9315b564bdedad672039306,-23.040252,-46.979782


- Je supprime les lignes où *customer_unique_id* est manquant car c'est une variable essentielle (c'est elle qu'on va regrouper en cluster)

In [13]:
order_items = order_items.loc[order_items['customer_unique_id'].isnull() == False]

In [14]:
order_items.isnull().sum()

order_id                           0
order_item_id                      0
product_id                         0
seller_id                          0
shipping_limit_date                0
price                              0
freight_value                      0
customer_id                        0
order_purchase_timestamp           0
delivery_time                      0
product_category_name_english      0
payment_type                       1
payment_installments               1
payment_value                      1
seller_lat                         0
seller_lng                         0
review_score                     646
customer_unique_id                 0
customer_lat                       0
customer_lng                       0
dtype: int64

### **Création d'une nouvelle variable *customer_seller_distance***

À partir de customer_lat/customer_lng et seller_lat/seller_lng, je vais calculer la distance client/vendeur pour chaque commande.

- Création d'une fonction qui va permettre de calculer la distance entre deux points à partir de leur coordonnées GPS : 

In [15]:
import math

def haversine_distance(coord1, coord2):

    """
    Cette fonction permet de calculer la distance entre un point A et un point B à partir de leur coordonnées GPS.

    :coord1: coordonnées du point A, sous forme de tuple (latitude, longitude).
    :coord2: coordonnées du point A, sous forme de tuple (latitude, longitude).
    :return: Retourne la distance de haversine entre A et B, en kilomètres.

    """

    lat1, lon1 = coord1
    lat2, lon2 = coord2
    R = 6371  # Rayon moyen de la Terre en kilomètres

    dlat = math.radians(lat2 - lat1)
    dlon = math.radians(lon2 - lon1)

    a = math.sin(dlat / 2) * math.sin(dlat / 2) + math.cos(math.radians(lat1)) * math.cos(math.radians(lat2)) * math.sin(dlon / 2) * math.sin(dlon / 2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = R * c
    return distance

- On applique la fonction haversine_distance() pour chaque ligne et on met le résultat dans la colonne *customer_seller_distance*

In [16]:
order_items['customer_seller_distance'] = order_items.apply(lambda row: haversine_distance((row['customer_lat'], row['customer_lng']), (row['seller_lat'], row['seller_lng'])), axis=1)

C:\Users\Data Science\AppData\Local\Temp\ipykernel_5604\347066452.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  order_items['customer_seller_distance'] = order_items.apply(lambda row: haversine_distance((row['customer_lat'], row['customer_lng']), (row['seller_lat'], row['seller_lng'])), axis=1)


- On va supprimer les colonnes dont on a plus besoin à ce stade : *product_id*, *seller_id*, *shipping_limit_date*, *customer_id*, *seller_lat*, *seller_lng* :

In [17]:
colonnes_a_supprimer = [
                        'order_item_id',
                        'product_id',
                        'seller_id',
                        'price',
                        'shipping_limit_date',
                        'customer_id',
                        'seller_lat',
                        'seller_lng'
                       ]

# On en profite pour créer un dataset global nommé "data"
data = order_items.drop(colonnes_a_supprimer, axis=1)

### **Regroupement des lignes pour n'avoir plus qu'une ligne par client**

- Je duplique la colonne *order_purchase_timestamp* : lors du groupby, une sera utilisé pour définir la date de première commande, l'autre pour la date de dernière commande.

In [18]:
data['order_purchase_timestamp_last'] = data.loc[:, 'order_purchase_timestamp']
# Je renomme la colonne order_purchase_timestamp d'origine en order_purchase_timestamp_first
data.rename(columns={"order_purchase_timestamp": "order_purchase_timestamp_first"}, inplace=True)

- Nous allons maintenant regrouper les lignes de manière à n'avoir plus qu'une ligne par *customer_unique_id*, c'est-à-dire par client.<br>
La stratégie de regroupement dépendra du type de colonne.

In [19]:
# Je définis la fonction first_mode() qui va me servir pour certaines variables catégorielles
def first_mode(series):
    """
    Cette fonction permet de ne retenir que la valeur la plus fréquente lors d'un groupby.
    Si plusieurs valeurs à égalité, alors ne retourne que la première valeur.
    """
    modes = series.mode()
    if not modes.empty:
        return modes.iloc[0]
    else:
        return None

# Je définis les stratégies de regroupement pour chaque colonne
strategie = {
             'order_id': 'count', # Combien le client a passé de commande au total
             'freight_value': 'sum', # Combien le client a payé de frais de port au total
             'order_purchase_timestamp_first': 'min', # Date de la première commande du client
             'delivery_time': 'mean', # En moyenne, le client se fait livrer en combien de temps
             'product_category_name_english': first_mode, # Valeur la plus fréquemment recontrée, si plusieurs à égalité, je ne prends que la première retournée
             'payment_type': first_mode, # Valeur la plus fréquemment recontrée, si plusieurs à égalité, je ne prends que la première retournée
             'payment_installments': 'mean', # En moyenne, le client paie en combien de fois
             'payment_value': 'sum', # Combien le client a dépensé au total
             'review_score': 'mean', # En moyenne, quelle note met le client
             'customer_lat': 'mean', # Je prends la moyenne, car le client peut avoir déménagé entre deux commandes
             'customer_lng': 'mean', # Je prends la moyenne, car le client peut avoir déménagé entre deux commandes
             'customer_seller_distance': 'mean', # En moyenne, le client est à quelle distance des/du vendeur(s)
             'order_purchase_timestamp_last': 'max' # Date de la dernière commande du client
            }

# Regroupement des lignes pour n'avoir plus qu'une ligne par client
data = data.groupby('customer_unique_id').agg(strategie)

data = data.reset_index() # pour remettre customer_unique_id en tant que colonne

- Traitements divers suite au regroupement : 

In [20]:
print(f"Avant traitement : {data.shape[0]} lignes, {data.shape[1]} colonnes.")

print("Il y a", data['customer_unique_id'].duplicated().sum(), "client(s) en double.")

# payment_type et payment_installments sont manquants sur une ligne : je supprime la ligne concernée
data = data.loc[data['payment_type'].isnull() == False]

print("Il y a", data['review_score'].isnull().sum(), "clients qui n'ont pas laissé d'avis, sur un total de", len(data), ".")
# Un nombre très réduit de client n'a jamais laissé d'avis suite à une commande.
# Étant donné qu'il est compliqué d'imputer ces valeurs manquantes sans perdre la notion de hiérarchie
# dans les notes moyenne et que très peu d'individus sont concernés,
# je décide de supprimer les clients qui n'ont jamais laissé un avis
data = data.loc[data['review_score'].isnull() == False]

# Renommage de certaines variables avec des noms plus explicites
a_renommer = {
              "order_id": "orders_number", # order_id représente maintenant le nombre de commandes pour chaque client
              "product_category_name_english": "preferred_category",
              "payment_type": "preferred_payment_type",
              "payment_value": "total_spent",
              "order_purchase_timestamp_first": "first_purchase_date",
              "order_purchase_timestamp_last": "last_purchase_date",
              "delivery_time": "avg_delivery_time",
              "payment_installments": "avg_payment_installments",
              "review_score": "avg_review_score",
              "customer_seller_distance": "avg_customer_seller_distance_km"
             }
data.rename(columns=a_renommer, inplace=True)

# Je vais créer une nouvelle variable correspondant à la proportion (en %) de frais de port dans le total dépensé
# cette variable sera plus pertinante que freight_value (total dépensé en frais de port par le client), que je vais supprimer
data['shipping_fees_proportion'] = data['freight_value']/data['total_spent']*100
data = data.drop(['freight_value'], axis=1)

# Je réarrange l'ordre des colonnes
ordre = [
         'customer_unique_id',
         'customer_lat',
         'customer_lng',
         'orders_number',
         'first_purchase_date',
         'last_purchase_date',
         'total_spent',
         'preferred_payment_type',
         'avg_payment_installments',
         'shipping_fees_proportion',
         'avg_customer_seller_distance_km',
         'avg_delivery_time',
         'preferred_category',
         'avg_review_score'         
        ]
data = data[ordre]

print(f"Après traitement : {data.shape[0]} lignes, {data.shape[1]} colonnes.")


Avant traitement : 93350 lignes, 14 colonnes.
Il y a 0 client(s) en double.
Il y a 603 clients qui n'ont pas laissé d'avis, sur un total de 93349 .
Après traitement : 92746 lignes, 14 colonnes.


In [21]:
data

,customer_unique_id,customer_lat,customer_lng,orders_number,first_purchase_date,last_purchase_date,total_spent,preferred_payment_type,avg_payment_installments,shipping_fees_proportion,avg_customer_seller_distance_km,avg_delivery_time,preferred_category,avg_review_score
0,0000366f3b9a7992bf8c76cfdf3221e2,-23.340235,-46.830140,1,2018-05-10 10:56:27,2018-05-10 10:56:27,141.90,credit_card,8.0,8.456660,110.568636,6.0,bed_bath_table,5.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,-23.559115,-46.787626,1,2018-05-07 11:11:27,2018-05-07 11:11:27,27.19,credit_card,1.0,30.489150,22.168333,3.0,health_beauty,4.0
2,0000f46a3911fa3c0805444483337064,-27.542880,-48.633426,1,2017-03-10 21:05:03,2017-03-10 21:05:03,86.22,credit_card,8.0,19.972164,516.938836,25.0,stationery,3.0
3,0000f6ccb0745a6a4b88665a16c9f078,-1.312214,-48.483159,1,2017-10-12 20:29:41,2017-10-12 20:29:41,43.62,credit_card,4.0,40.417240,2481.287188,20.0,telephony,4.0
4,0004aac84e0df4da2b147fca70cf8255,-23.505548,-47.469705,1,2017-11-14 19:45:42,2017-11-14 19:45:42,196.89,credit_card,6.0,8.578394,154.507887,13.0,telephony,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93345,fffcf5a5ff07b0908bd4e2dbc735a684,-8.362511,-36.562343,1,2017-06-08 21:00:36,2017-06-08 21:00:36,2067.42,credit_card,10.0,12.029970,1996.470471,27.0,health_beauty,5.0
93346,fffea47cd6d3cc0a88bd621562a9d061,-12.217395,-38.959308,1,2017-12-10 20:07:56,2017-12-10 20:07:56,84.58,credit_card,1.0,23.279735,1527.201720,30.0,baby,4.0
93347,ffff371b4d645b6ecea244b27531430a,-11.834552,-55.508110,1,2017-02-07 15:49:16,2017-02-07 15:49:16,112.46,credit_card,1.0,20.060466,1526.116170,14.0,auto,5.0
93348,ffff5962728ec6157033ef9805bacc48,-21.126568,-41.672806,1,2018-05-02 15:17:41,2018-05-02 15:17:41,133.69,credit_card,5.0,13.980103,637.858806,11.0,watches_gifts,5.0


In [22]:
data.isnull().sum()

customer_unique_id                 0
customer_lat                       0
customer_lng                       0
orders_number                      0
first_purchase_date                0
last_purchase_date                 0
total_spent                        0
preferred_payment_type             0
avg_payment_installments           0
shipping_fees_proportion           0
avg_customer_seller_distance_km    0
avg_delivery_time                  0
preferred_category                 0
avg_review_score                   0
dtype: int64